### Using PuLP package

In [5]:
import pandas as pd
import pulp
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

In [ ]:
class MaintenanceOptimization:
    '''
    data: 
        The dataframe containing turbine fault, SCADA, and status information.
    model: 
        The optimization model for minimizing maintenance costs.
    downtime_vars: 
        A dictionary of decision variables representing downtime hours for each observation.
    '''
    def __init__(self, data):
        self.data = data
        self.model = LpProblem("Maintenance_Cost_Minimization", LpMinimize)
        self.downtime_vars = LpVariable.dicts("Downtime", data.index, lowBound=0, upBound=24, cat='Continuous')

    def setup_objective(self):
        '''
        Sets up the objective function for the optimization model, which minimizes total maintenance and downtime costs.
        '''
        maintenance_costs = []
        downtime_costs = []

        for i, row in self.data.iterrows():
            fault_type = row['Fault']
            is_high_demand = row['Is_High_Demand']

            # Maintenance cost per fault type
            if is_high_demand:
                maintenance_cost = 150000
            elif fault_type == 'Routine':
                maintenance_cost = 750000
            else:
                maintenance_cost = 50000

            # Downtime cost
            production_loss = row['WEC: Production kwh'] * self.downtime_vars[i]
            downtime_cost = production_loss * 0.1  # Cost per kWh downtime

            maintenance_costs.append(maintenance_cost)
            downtime_costs.append(downtime_cost)

        # Define objective function
        self.model += lpSum(maintenance_costs) + lpSum(downtime_costs), "Total_Maintenance_Cost"

    def add_constraints(self):
        '''
        Adds necessary constraints to the model, such as maximum allowable downtime per observation.
        '''
        # Add constraints (example: operational availability)
        for i in self.data.index:
            downtime_constraint = self.downtime_vars[i] <= 24
            constraint_name = f"Maximum allowable downtime for observation {i}"
            self.model += downtime_constraint, constraint_name

    def solve(self):
        '''
        Solves the optimization model and returns the decision variable values.
        '''
        self.model.solve()
        
        result_dict = {}
        for v in self.model.variables():
            result_dict[v.name] = v.varValue

        return result_dict

